In [1]:
import torch
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import os

In [2]:
def show_files(folder):
    for ind in range(len(folder)):
        print(f"{ind:3}: {folder[ind]}")

In [3]:
img_folder = "./images/"
arr = os.listdir(img_folder)
arr = [f for f in arr if os.path.isfile(f"{img_folder}{f}")]
arr.sort()
show_files(arr)

  0: 20220420_180907.jpg
  1: baby.jpg
  2: cafezin.jpg
  3: chuva.jpg
  4: end.jpg
  5: end_cervejada.jpg
  6: fatos.png
  7: festa_espelho.jpg
  8: fred.jpg
  9: gato_move.jpg
 10: gato_rack.jpg
 11: gato_tv.jpg
 12: jojo.jpg
 13: o_palhaco.jpg
 14: rick_chuva.jpg
 15: rick_end_cervejada.jpg
 16: rick_end_fantasia.jpg
 17: rieck.jpg
 18: serec.jpg
 19: serio.jpg
 20: simba_intenso.jpg
 21: sky.jpg
 22: veloso.jpg
 23: vlad.jpg


In [4]:
img_paths = []
for ind in range(len(arr)):
    path = f"{img_folder}{arr[ind]}"
    img_paths.append(path)

show_files(img_paths)

  0: ./images/20220420_180907.jpg
  1: ./images/baby.jpg
  2: ./images/cafezin.jpg
  3: ./images/chuva.jpg
  4: ./images/end.jpg
  5: ./images/end_cervejada.jpg
  6: ./images/fatos.png
  7: ./images/festa_espelho.jpg
  8: ./images/fred.jpg
  9: ./images/gato_move.jpg
 10: ./images/gato_rack.jpg
 11: ./images/gato_tv.jpg
 12: ./images/jojo.jpg
 13: ./images/o_palhaco.jpg
 14: ./images/rick_chuva.jpg
 15: ./images/rick_end_cervejada.jpg
 16: ./images/rick_end_fantasia.jpg
 17: ./images/rieck.jpg
 18: ./images/serec.jpg
 19: ./images/serio.jpg
 20: ./images/simba_intenso.jpg
 21: ./images/sky.jpg
 22: ./images/veloso.jpg
 23: ./images/vlad.jpg


In [5]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5l6', pretrained=True)
# model.cuda()

results = model(img_paths, size=1280) # múltiplo de 32

Using cache found in /home/rick/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-5-19 Python-3.11.3 torch-2.0.1+cpu CPU

Fusing layers... 


requirements: /home/rick/Documents/University/TCC/TCC/lib/python3.11/site-packages/requirements.txt not found, check failed.


YOLOv5l6 summary: 476 layers, 76726332 parameters, 0 gradients
Adding AutoShape... 


In [6]:

tags = results.pandas().xyxy[7][["name"]].groupby("name").value_counts()
tag_list = list(zip(tags.index, tags))
tag_list[0]

res = pd.concat(results.pandas().xyxy)
tags_res = res[['name']].groupby('name').value_counts().index
tags_res = list(tags_res)
tags_res.append('group')
tags_res.append('trio')
tags_res.append('pair')
tags_res.sort()

tags_res

['backpack',
 'bed',
 'bench',
 'book',
 'bottle',
 'bus',
 'cake',
 'cat',
 'cell phone',
 'chair',
 'cup',
 'dining table',
 'dog',
 'group',
 'handbag',
 'pair',
 'person',
 'remote',
 'tie',
 'toilet',
 'trio',
 'tv']

In [7]:
DG = nx.DiGraph()

In [8]:
for i in range(len(arr)):
    tag = results.pandas().xyxy[i][["name"]].groupby("name").value_counts()
    tags = list(zip(tag.index, tag))

    img_name = arr[i]

    for j in range(len(tags)):
        tag_name = tags[j][0]
        tag_quant = tags[j][1]
        
        DG.add_edge(tag_name, img_name) # ligando nome da classe identificada com o nome do arquivo
        if tag_name == 'person':
            if tag_quant == 2:
                DG.add_edge('pair', img_name)
            elif tag_quant == 3:
                DG.add_edge('trio', img_name)
            if tag_quant > 3:
                DG.add_edge('group', img_name)

In [9]:
nx.draw_shell(DG, with_labels=True, font_weight='bold')

print(DG.number_of_nodes())
print(DG.number_of_edges())

plt.show()
plt.savefig("results/path.png")
plt.clf()

44
62


In [10]:
pos = nx.bipartite_layout(DG, tags_res, aspect_ratio=9/16)
nx.draw(DG, pos, with_labels=True, font_weight='bold')
plt.savefig('results/columns.png')
plt.clf()